In [14]:
import pandas as pd
import requests
import time
import json

In [2]:
data= pd.read_csv('/..../Word counts.csv')
data.head()

,Name,Count
0,Aaptos aaptos,244
1,Abacarus,364
2,Abacarus hystrix,258
3,Abacarus panticis,0
4,Abalistes,3752


### clean the data

In [4]:
filter1= data['Count']!= '0'
filter2= data['Count']!= 'count field absent'
cleanData= data[filter1 & filter2]
cleanData.head()

,Name,Count
0,Aaptos aaptos,244
1,Abacarus,364
2,Abacarus hystrix,258
4,Abalistes,3752
5,Abarema,9250


#### Define a function to get data about a name from the API

In [ ]:
def getdata(word):
    temp= word
    tempdata= None
    parameters= {'name': temp}
    try:
        response= requests.get('http://api.gbif.org/v1/species', params= parameters)
    except Exception as error:
        time.sleep(5)
        print(error)
        tempdata= getdata(temp)
    if not tempdata:
        tempdata= json.loads(response.content)
    return tempdata

#### Create a dataframe AgSpecies: Stores the name, rank and class of all "species".

In [ ]:
AgSpecies= pd.DataFrame({'name': [], 'rank': [], 'class': []})
for i in list(cleanData.index):
    apiData= getdata(cleanData['Name'][i])
    if apiData['results']== []:
        continue
    elif 'rank' in list(apiData['results'][0].keys()) and 'class' in list(apiData['results'][0].keys()):
        if apiData['results'][0]['rank']== 'SPECIES':
            AgSpecies= AgSpecies.append({'name': cleanData['Name'][i], 'rank': apiData['results'][0]['rank'], 'class': apiData['results'][0]['class']}, ignore_index= True)
            print(i)

In [ ]:
AgSpecies.to_csv('/....../Species_count.csv')